# 03 - Interactive Viz

## Deadline

Wednesday November 8th, 2017 at 11:59PM

## Important Notes

- Make sure you push on GitHub your Notebook with all the cells already evaluated
- Note that maps do not render in a standard Github environment : you should export them to HTML and link them in your notebook.
- Remember that `.csv` is not the only data format. Though they might require additional processing, some formats provide better encoding support.
- Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
- Please write all your comments in English, and use meaningful variable names in your code

## Background

In this homework we will be exploring interactive visualization, which is a key ingredient of many successful data visualizations (especially when it comes to infographics).

Unemployment rates are major economic metrics and a matter of concern for governments around the world. Though its definition may seem straightforward at first glance (usually defined as the number of unemployed people divided by the active population), it can be tricky to define consistently. For example, one must define what exactly unemployed means : looking for a job ? Having declared their unemployment ? Currently without a job ? Should students or recent graduates be included ? We could also wonder what the active population is : everyone in an age category (e.g. `16-64`) ? Anyone interested by finding a job ? Though these questions may seem subtle, they can have a large impact on the interpretation of the results : `3%` unemployment doesn't mean much if we don't know who is included in this percentage. 

In this homework you will be dealing with two different datasets from the statistics offices of the European commission ([eurostat](http://ec.europa.eu/eurostat/data/database)) and the Swiss Confederation ([amstat](https://www.amstat.ch)). They provide a variety of datasets with plenty of information on many different statistics and demographics at their respective scales. Unfortunately, as is often the case is data analysis, these websites are not always straightforward to navigate. They may include a lot of obscure categories, not always be translated into your native language, have strange link structures, â€¦ Navigating this complexity is part of a data scientists' job : you will have to use a few tricks to get the right data for this homework.

For the visualization part, install [Folium](https://github.com/python-visualization/folium) (*HINT*: it is not available in your standard Anaconda environment, therefore search on the Web how to install it easily!). Folium's `README` comes with very clear examples, and links to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find two `.topojson` files, containing the geo-coordinates of 

- European countries (*liberal definition of EU*) (`topojson/europe.topojson.json`, [source](https://github.com/leakyMirror/map-of-europe))
- Swiss cantons (`topojson/ch-cantons.topojson.json`) 

These will be used as an overlay on the Folium maps.


In [2]:
import os
import pandas as pd
import json
import folium
import re
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

# Assignment
## Question 1
 Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.



## Choice of the dataset
lfsq_urgan 


In [3]:
rate_filepath='./dataEurope/lfsa_urgan.tsv'
#number_filepath='./dataEurope/lfsa_ugan.tsv'

def import_file(filepath):
    df2=pd.read_table(filepath)
    s=df2[df2.columns[0]]
    df1=s.str.split(pat=',', expand=True)
    df1.columns=('Unit', 'Sex', 'Age', 'Citizen', 'Country')
    df2.drop(df2.columns[0],axis=1,inplace=True)
    full_df=pd.concat([df1,df2], axis=1)
    #Cleaning the series for the most recent entry (2016)
    s_lastyear=full_df[full_df.columns[5]]
    for idx in s_lastyear.index:
        non_decimal = re.compile(r'[^\d.]+')
        s_lastyear[idx]=non_decimal.sub('',s_lastyear[idx])
        try:
            s_lastyear[idx]=float(s_lastyear[idx])
        except:
            s_lastyear[idx]=np.nan
    return pd.concat([full_df[['Unit', 'Sex', 'Age', 'Citizen', 'Country']],s_lastyear],axis=1)

df_rate=import_file(rate_filepath)
df_rate.rename(columns={df_rate.columns[5]:'Rate'}, inplace=True)
#df_rate.Country

In [4]:
c=df_rate.Country.unique()
c.size

39

In [5]:
#df_both=pd.concat([df_rate,df_numb[df_numb.columns[5]]],axis=1)
cropped1=df_rate[df_rate.Age=='Y15-74']
cropped2=cropped1[cropped1.Sex=='T']
cropped3=cropped2[cropped2.Citizen=='TOTAL']
cropped3['Rate']
test=pd.DataFrame({'Country': ['FR', 'CH'], 'Rate':[18,2]})
c=cropped3.Country
c.values.size

39

In [34]:
m=folium.Map(location=[22.3, 6.66])
path_EU='./topojson/europe.topojson.json'
eu_json_data=json.load(open(path_EU))
eu_map=folium.Map(location=[46, 7],zoom_start=3)
eu_map.choropleth(geo_data=eu_json_data,topojson='objects.europe', data=cropped3, columns=['Country', 'Rate'],
                 key_on='feature.id', fill_color='YlGn', fill_opacity=0.8, line_opacity=0.6,
             legend_name='Unemployement rate(%)')
#eu_map


## Question 2
Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

In [87]:
path='./dataCH/TauxDeChomageCH.csv'
df=pd.read_csv(path, thousands='\'',encoding='utf_16le', 
index_col=['Canton'])

df.drop('Grande région',axis=1,inplace=True)
df.drop('Mois', axis=1, inplace=True)
df.rename(columns={df.columns[1]:'Grande région'}, inplace=True)
df

,Région linguistique,Grande région,Janvier 2017,Janvier 2017.1,Janvier 2017.2,Janvier 2017.3,Février 2017,Février 2017.1,Février 2017.2,Février 2017.3,...,Septembre 2017.2,Septembre 2017.3,Octobre 2017,Octobre 2017.1,Octobre 2017.2,Octobre 2017.3,Total,Total.1,Total.2,Total.3
Canton,,,,,,,,,,,,,,,,,,,,,
,,NaN,Taux de chômage,Chômeurs inscrits,Demandeurs d'emploi,Demandeurs d'emploi non chômeurs,Taux de chômage,Chômeurs inscrits,Demandeurs d'emploi,Demandeurs d'emploi non chômeurs,...,Demandeurs d'emploi,Demandeurs d'emploi non chômeurs,Taux de chômage,Chômeurs inscrits,Demandeurs d'emploi,Demandeurs d'emploi non chômeurs,Taux de chômage,Chômeurs inscrits,Demandeurs d'emploi,Demandeurs d'emploi non chômeurs
Berne,Suisse alémanique,Espace Mittelland,3,16'954,21'586,4'632,3,16'738,21'367,4'629,...,18'385,4'727,0,0,0,0,2.4,134'156,176'193,42'037
Soleure,Suisse alémanique,Espace Mittelland,3.2,4'744,7'309,2'565,3.2,4'602,7'260,2'658,...,6'628,2'827,0,0,0,0,2.6,37'635,62'173,24'538
Bâle-Ville,Suisse alémanique,Nordwestschweiz,4.2,4'181,5'954,1'773,4.1,4'075,5'901,1'826,...,5'168,1'713,0,0,0,0,3.4,33'939,49'953,16'014
Bâle-Campagne,Suisse alémanique,Nordwestschweiz,3.1,4'659,6'111,1'452,3.1,4'656,6'129,1'473,...,5'540,1'458,0,0,0,0,2.6,39'003,52'285,13'282
Argovie,Suisse alémanique,Nordwestschweiz,3.5,12'622,17'038,4'416,3.4,12'551,16'970,4'419,...,15'145,4'461,0,0,0,0,2.8,103'522,144'005,40'483
Zurich,Suisse alémanique,Zürich,3.9,32'387,39'340,6'953,3.9,31'619,38'748,7'129,...,34'156,6'931,0,0,0,0,3.2,263'669,327'477,63'808
Glaris,Suisse alémanique,Ostschweiz,2.5,567,876,309,2.6,576,891,315,...,713,297,0,0,0,0,2,4'402,7'080,2'678
Schaffhouse,Suisse alémanique,Ostschweiz,3.6,1'587,2'653,1'066,3.7,1'608,2'676,1'068,...,2'328,1'042,0,0,0,0,3,12'898,22'810,9'912


## Question 3
 Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.


## Question 4 (Bonus)

 *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [RÃ¶stigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?